plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Input symptom, mode, perdur
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat

#
import pandas as pd
import datetime
# from plot_change_module import *

## Define Functions

In [2]:
def open_csv(mat_file):
    
    with open(mat_file,'r') as sz_mat:
        mat_csv_list = list(csv.reader(sz_mat))

        all_sx = mat_csv_list[0] #collect symptom abbreviations
        sx_col = all_sx.index(sx_input) #find col num of symptom
        csv_rows = np.size(mat_csv_list,0) #find row num 

        for r in range(1,csv_rows):  #collect symptom data for specfic seizure in list
            sx_data.append(mat_csv_list[r][sx_col]) 

        try: 
            mx_index = sx_data.index(mx_input) #find row num of first occurance of mode  
            return mx_index

        except ValueError:
            print('no matching mode')
            

In [3]:
def open_xl(xl_name): 
    # df = pd.read_excel(xl_name)
    # Ndimensions = df.shape #row, col
    # params_list = df.values.tolist()
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [4]:
def ll_weights(sz_name,perdur_input):
    print(sz_name)

    sz_path = pt_path + sz_data[pt_i]
    os.chdir(sz_path)

    df = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input])
    try:
        mx_index = df.index[df[sx_input]==int(mx_input)]
        # first_mx = mx_index[0]
        # print(first_mx)
        # print((mx_index[0])/5)
    except IndexError:
        print('n/a')
    
    #load fs, ppEEG, study

    sz_mat = loadmat(sz_name +'.mat')      
    ppEEG = sz_mat['ppEEG']

    try:
        fs = [[element for element in upperElement] for upperElement in sz_mat['fs']]

        ppEEG_col = len(sz_mat['ppEEG'][0])
        ppEEG_row = len(sz_mat['ppEEG'])
        ppEEG_mx = round(((mx_index[0])/5)*fs[0][0])
        print(ppEEG_mx)
        ppEEG_perdur = int(perdur_input) * fs[0][0]

        # print(ppEEG_row,ppEEG_col)
        # print(ppEEG_mx)
        # print(ppEEG_perdur)
        ppEEG_onset_list = []
        ppEEG_before_list = []
        ppEEG_after_list = []
        
        try:
            for p in range(0,ppEEG_row-1):
                ppEEG_onset_list.append(ppEEG[p][ppEEG_mx]) #at exact onset
                ppEEG_before_list.append(ppEEG[p][ppEEG_mx-ppEEG_perdur])  #perdur before
                ppEEG_after_list.append(ppEEG[p][ppEEG_mx+ppEEG_perdur])   #perdur after
            return ppEEG_onset_list,ppEEG_before_list,ppEEG_after_list
        except:
            print('no ppEEG list')

    except:
        print('no fs or ppEEG')

## Input symptom, mode, and duration

In [5]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

## Load paramaters

In [6]:
opscea_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/' # set path
os.chdir(opscea_path)

df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

## Load list of patient and seizure names

In [7]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'
os.chdir(avg_change_path)

sz_name_nd = open_xl('sz_name_list.xlsx')
with open('sz_name_list.csv','r') as sz_names:
    sz_names_list = list(csv.reader(sz_names))
    sz_rows = np.size(sz_names_list,0) #find row num 
    pt_data = []
    sz_data= []
    for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
        pt_data.append(sz_names_list[r][0]) 
        sz_data.append(sz_names_list[r][1]) 

## For loop through patients in list matching seizure

In [8]:
for pt in pt_data:
    
    pt_i = pt_data.index(pt)
    sz_name = sz_data[pt_i]

    pt_path = avg_change_path + pt + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)
    
    for sz_name in sz_data:
        
        if sz_name in pt_dir:
            ll = ll_weights(sz_name,perdur_input)
            try:
                ll_onset = ll[0] #onset
                ll_before = ll[1] #before
                ll_after = ll[2] #after
                
            except:
                print('no ll list')
            
            os.chdir(pt_path + 'Imaging/elecs')
            e_mat = loadmat('clinical_elecs_all.mat')
            anatomy = e_mat['anatomy']
            em = e_mat['elecmatrix']
            
            #extract before and after from ll_weights

        #load electrodes -- anatomy, eleclabels, elecmatrix
        # os.chdir(path3)

EC96_01
49459
[219.5354756869439, -113.79545098519063, 423.2951436718172, 31.506119759912277, 382.9390703129598, 72.84301915160864, -217.756382398634, -32.00133754360946, nan, -160.4594333894656, 241.61705973861692, -163.00974623092407, 89.10543094864984, 133.40499834263755, 141.746569206117, 63.114299721088905, -539.1775535178383, -792.0676279898562, -168.96731798900055, -46.656570656084945, 191.95298855361173, 175.62767030855463, -129.09407969969055, 192.90925933301153, -703.2836873375273, -659.1810670039326, 113.0518220185892, 133.9777713525882, 179.55583487688418, 138.27888353381437, 100.39172091911819, 124.06646175147371, -420.0134156221085, 55.675540676461964, -75.85103522989178, 123.09552659498729, 158.57319416061515, 229.75513609133534, 168.6812317021923, 77.33728816992131, -457.53757343776124, 458.1369646050004, -115.50403485211046, 57.05610404919726, 155.10130533714715, 155.70980698591168, 193.16377277430365, 159.7987446179027, -499.3806716510839, -43.01045116983865, -512.584

## Load frame speed and ppEEG

In [9]:
        # path2 = os.path.join(path1,sz_name)
        # os.chdir(pt_path)
        # elecs_path = os.path.join(pt_path,'Imaging/elecs')
        # os.chdir(elecs_path)


## Collect linelength weight times

In [10]:
#load sfx from .mat
seconds = (first_mx / 5 * sfx)


NameError: name 'first_mx' is not defined

In [ ]:
            #COLLECT LL WEIGHT TIMES 
            
            #find first occurance of either 1, 2, or 3 
#             td_perdur = pd.Timedelta(int(perdur_input), "s")
            
    
#             ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first)
#             ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first - td_perdur)
#             ll_weight_times.append(pd.Timedelta(mx_index / 5) + sem_first + td_perdur)

            #collect sorting index
            #collect relevant snippet from ll based on ll_weight_times
            

## Load semiology matrix

In [ ]:
            # print(df)
            # time_mat_file = '_'.join([sz_name,'time_mat.csv'])
            

#             with open(time_mat_file,'r') as sz_time_mat:
                
#                 time_mat_csv_list = list(csv.reader(sz_time_mat))

#                 vid_start = time_mat_csv_list[1][0]
#                 vid_end = time_mat_csv_list[1][1]
#                 mat_start = time_mat_csv_list[1][2]
#                 mat_end = time_mat_csv_list[1][3]

#                 td_vid_start = pd.Timedelta(vid_start)
#                 td_vid_end = pd.Timedelta(vid_end)


#                 td_mat_start = pd.Timedelta(mat_start)
#                 td_mat_end = pd.Timedelta(mat_end)

#                 td_vid_dur = td_vid_end - td_vid_start
#                 # print(td_vid_dur)
#                 td_mat_dur = td_mat_end - td_mat_start
#                 # print(td_mat_dur)


#                 #if params vid start == start of video
#             # print(params_list[num][:])
#     #                 td_params_stop = pd.Timedelta(int(params_vid_stop), "s")


#     #                 short_vid_start = td_mat_start + td_mat_dur - td_params_stop
#                 sem_first = td_mat_start - td_vid_start
#                 sem_last = sem_first + td_mat_dur
#                 sem_period = [sem_first, sem_last]


#                 #convert sem_first in m:s.ms --> # of 200 ms 
#                 #pad with sem_first 
#                 pad_start_ms = sem_first.total_seconds() #* 1000 
#                 print(sz_name)
                
#                 print('col num:', df.size) #size should be vid dur + 1

#                 print('vid dur in 200 ms:', (td_vid_dur).total_seconds()* 5 )

#                 print('mat in 200 ms:', (td_mat_dur).total_seconds()*5)
#                 # print('mat dur:', (td_mat_dur).total_seconds()*1000/5)
#                 print('pad start with:', pad_start_ms*5)
                
# #                 x = np.full([1,2,3],np.nan)
# #                 x = np.empty(int(pad_start_ms/5))
# #                 x.fill(np.nan)
# #                 print(int(pad_end_ms/5))

# #                 print(x)


#             # x = nan(pad_start_ms/5)

#                 if td_mat_end < td_vid_end:
#                     td_pad_end = td_vid_end - td_mat_end
#                     pad_end_ms = td_pad_end.total_seconds() #* 1000
#                     print('pad end with:', pad_end_ms*5)

#                 elif td_mat_end > td_vid_end:
#                     td_trim_end = td_mat_end - td_vid_end
#                     trim_end_ms = td_trim_end.total_seconds() #* 1000

#                     print('trim end by:', trim_end_ms*5)
#                 elif td_vid_end == td_mat_end:
#                     print(td_vid_end)
#                     print(td_mat_end)

## Find first occurance of mode

In [ ]:
#make applicable to multiple seizures from single patient        
        # params_vid_start = df_params.loc['EC96']['VIDstart']
        # print(params_vid_start)
        # params_vid_stop = df_params.loc['EC96']['VIDstop']
        # print(params_vid_stop)

Extract parameters

Set path and initialize variables for semiology and time matrix

In [ ]:
            

        #load symptom matrix (mat.csv) and time matrix (time_mat.csv) and electrode info over time matrix (sz.mat)
            # os.chdir(path2)

            # mat_file = '_'.join([sz_name,'mat.csv'])


Load time matrix and match up with video and semiology matrix

In [ ]:
            
#             elecs_mat = loadmat('clinical_elecs_all.mat')            
        
#                 #load anatomy
#             anatomy = [[element for element in upperElement] for upperElement in elecs_mat['anatomy']]
            # eleclabels = [[element for element in upperElement] for upperElement in elecs_mat['eleclabels']]
            # elecmatrix = [[element for element in upperElement] for upperElement in elecs_mat['elecmatrix']]
            
            
            
            #TRY/EXCEPT INSTEAD
            # make sure variables are present
#             if 'anatomy' in locals() or 'anatomy' in globals():
#                 #collect neuroanatomy by row
#                 for a_row in range(0,np.size(anatomy,0)):
#                     if np.size(anatomy[a_row][3]) == 1:
#                         anat.append(anatomy[a_row][3][0])
#                 del anatomy

                